<a href="https://colab.research.google.com/github/willianszwy/Aprendizado-Supervisionado-no-Neuronio-Perceptron/blob/main/Perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import random

# Parte I - Resolvendo um problema linearmente separável

In [ ]:
random.seed(323452345)

Carregando os exemplos a partir de um repositório do github separado

In [1]:
!git clone https://github.com/jesusdourado/rna-atividade2.1-data.git

Cloning into 'rna-atividade2.1-data'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 0), reused 9 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [4]:

exemplos = np.fromfile('rna-atividade2.1-data/data/dataAll.txt').reshape(-1,3)


## Implementando o neurônio Perceptron de Rosenblatt

In [5]:
class Perceptron:

    def __init__(self, tamanho, bias = 0, taxa_aprendizado= 0.1) -> None:
        self.bias = bias
        self.taxa_aprendizado = taxa_aprendizado
        self.pesos = np.zeros(tamanho + 1)
        self.pesos[0] = random.uniform(-0.5,0.5)
        self.pesos[1] = random.uniform(-0.5,0.5)
        self.pesos[2] = random.uniform(-0.5,0.5)
        self.entradas = np.empty(tamanho + 1)
        self.entradas[0] = self.bias
        self.erro = 0
        self.atualiza_pesos_contador = 0 


    def treinar(self, d):
        epoca = 0
        continua = True
        while continua:
            epoca += 1
            print("Época: ", epoca)
            contem_erro = False
            qtd_ajustes_pesos_epoca = 0

            for i in range(d.shape[0]):

                self.entradas[1] = d[i][0]
                self.entradas[2] = d[i][1]

                y = self.predicao()
                self.calcula_erro(y, d[i][2])

                if self.erro != 0:
                    self.atualiza_pesos()
                    qtd_ajustes_pesos_epoca += 1
                    contem_erro = True      

            print("Total ajustes de pesos da época: ", qtd_ajustes_pesos)

            if not contem_erro:
                continua = False
        
        print("Número total de ajustes dos pesos: ", self.atualiza_pesos_contador)
        print("Número de Épocas: ", epoca)

    def funcao_de_ativacao(self, x):
        return 1 if x >= 0 else 0

    def predicao(self):
        y = self.pesos.T.dot(self.entradas)
        return self.funcao_de_ativacao(y)

    def calcula_erro(self,y, yd):
        self.erro = yd - y

    def delta(self):
        return self.taxa_aprendizado * self.erro * self.entradas

    def atualiza_pesos(self):
        
            self.pesos =  self.pesos + self.delta()
            self.atualiza_pesos_contador += 1
            print("pesos atualizados: ",self.pesos)

In [6]:
neuronio = Perceptron(2)
neuronio.treinar(exemplos)

Época:  1
pesos atualizados:  [  0.14081284  36.08338618 -24.2618309 ]
pesos atualizados:  [   0.14081284  -17.57566382 -100.7575809 ]
pesos atualizados:  [  0.14081284  55.22035618 -64.9280209 ]
pesos atualizados:  [  0.14081284 137.81905618  14.9278091 ]
pesos atualizados:  [ 1.40812843e-01  1.43664816e+02 -6.33667809e+01]
pesos atualizados:  [ 1.40812843e-01  1.03655906e+02 -1.41259471e+02]
pesos atualizados:  [ 1.40812843e-01  1.57325716e+02 -9.21672009e+01]
pesos atualizados:  [ 1.40812843e-01  1.04365656e+02 -1.49036661e+02]
pesos atualizados:  [ 1.40812843e-01  1.63925176e+02 -9.89143309e+01]
pesos atualizados:  [ 1.40812843e-01  1.18400546e+02 -1.64550261e+02]
pesos atualizados:  [ 1.40812843e-01  2.09116406e+02 -8.98369409e+01]
pesos atualizados:  [ 1.40812843e-01  1.51335416e+02 -1.50145091e+02]
pesos atualizados:  [ 1.40812843e-01  1.03068856e+02 -1.98614811e+02]
pesos atualizados:  [ 1.40812843e-01  1.73516206e+02 -1.56498541e+02]
pesos atualizados:  [ 1.40812843e-01  1.266

NameError: ignored